In [16]:
import sys

sys.path.append("..")

In [17]:
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torchvision.transforms import ToTensor
from torchvision.utils import draw_segmentation_masks
from tqdm import tqdm
from src.data import parse_annotations
from src.models import ResUNet

In [18]:
df = pd.read_csv("../data/hubmap/tile_meta.csv")
df.head()

,id,source_wsi,dataset,i,j
0,0006ff2aa7cd,2,2,16896,16420
1,000e79e206b7,6,3,10240,29184
2,00168d1b7522,2,2,14848,14884
3,00176a88fdb0,7,3,14848,25088
4,0033bbc76b6b,1,1,10240,43008


In [19]:
image_ids = df.loc[df["dataset"] == 1, "id"]
image_ids

4       0033bbc76b6b
16      00656c6f2690
17      0067d5ad2250
33      00d75ad65de3
34      00da70813521
            ...     
6844    f86347534ec1
6895    faba1bf818ae
6933    fc6def641612
6951    fd2437954fd8
6990    fe248458ea89
Name: id, Length: 422, dtype: object

In [20]:
polygons = pd.read_json("../data/hubmap/polygons.jsonl", lines=True).set_index("id")
polygons.head()

,annotations
id,
0006ff2aa7cd,"[{'type': 'glomerulus', 'coordinates': [[[167,..."
00168d1b7522,"[{'type': 'glomerulus', 'coordinates': [[[511,..."
0033bbc76b6b,"[{'type': 'blood_vessel', 'coordinates': [[[16..."
003504460b3a,"[{'type': 'blood_vessel', 'coordinates': [[[40..."
004daf1cbe75,"[{'type': 'blood_vessel', 'coordinates': [[[14..."


In [21]:
polygons = polygons.loc[image_ids, "annotations"]
polygons

id
0033bbc76b6b    [{'type': 'blood_vessel', 'coordinates': [[[16...
00656c6f2690    [{'type': 'blood_vessel', 'coordinates': [[[51...
0067d5ad2250    [{'type': 'unsure', 'coordinates': [[[90, 38],...
00d75ad65de3    [{'type': 'blood_vessel', 'coordinates': [[[38...
00da70813521    [{'type': 'blood_vessel', 'coordinates': [[[37...
                                      ...                        
f86347534ec1    [{'type': 'blood_vessel', 'coordinates': [[[42...
faba1bf818ae    [{'type': 'unsure', 'coordinates': [[[92, 408]...
fc6def641612    [{'type': 'blood_vessel', 'coordinates': [[[66...
fd2437954fd8    [{'type': 'unsure', 'coordinates': [[[378, 425...
fe248458ea89    [{'type': 'blood_vessel', 'coordinates': [[[13...
Name: annotations, Length: 422, dtype: object

In [22]:
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, poly_jsonl: pd.Series, image_dir: str = "../data/hubmap/train"):
        self.data = poly_jsonl
        self.root = Path(image_dir)
        self.to_tensor = ToTensor()
        assert self.root.exists() and self.root.is_dir()

    def __getitem__(self, idx):
        image_id = self.data.index[idx]
        image_path = self.root / f"{image_id}.tif"
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.to_tensor(image)

        ann = parse_annotations(self.data.iloc[idx], ("glomerulus",))
        masks = ann["glomerulus"]
        if masks.size > 0:
            mask = masks.sum(0).clip(0, 1).astype(np.uint8)
            mask = torch.tensor(mask, dtype=torch.uint8)
        else:
            mask = torch.zeros((512, 512), dtype=torch.uint8)

        return image, mask

    def __len__(self):
        return len(self.data)

In [23]:
dset = SegmentationDataset(polygons)

In [24]:
model = ResUNet(pretrained=False)
model.load_state_dict(torch.load("../models/cur_best/u-net.pth"))
model = model.to("cuda")
model.eval();

In [25]:
def process_mask(mask: torch.Tensor) -> torch.Tensor:
    assert mask.ndim == 2 and mask.dtype == torch.uint8
    mask = mask.detach().cpu().numpy()
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=20)
    return torch.tensor(mask)


In [26]:
inter_sum = 0
union_sum = 0
prediction_sum = 0
target_sum = 0

for image, mask in tqdm(dset):
    with torch.no_grad():
        image = image.to("cuda")
        pred = torch.sigmoid(model(image.unsqueeze(0))[0, 0]).cpu()
        pred = pred.round().to(torch.uint8)

        pred = process_mask(pred)

    pred_b = pred.to(torch.bool)
    mask_b = mask.to(torch.bool)
    inter_sum += torch.logical_and(pred_b, mask_b).sum()
    union_sum += torch.logical_or(pred_b, mask_b).sum()
    prediction_sum += pred_b.sum()
    target_sum += mask_b.sum()

100%|██████████| 422/422 [00:29<00:00, 14.38it/s]


In [27]:
print(f"IOU: {inter_sum / union_sum}")
print(f"Precision: {inter_sum / prediction_sum}")
print(f"Recall: {inter_sum / target_sum}")

IOU: 0.8623385429382324
Precision: 0.9767596125602722
Recall: 0.8804026246070862


| Processing       |  IOU  | Precision | Recall |
|------------------|-------|-----------|--------|
| None             | 0.820 |   0.910   |  0.892 |
| Closing, 7 x 2   | 0.818 |   0.905   |  0.896 |
| Closing, 7 x 10  | 0.796 |   0.868   |  0.906 |
| Closing, 7 x 20  | 0.769 |   0.830   |  0.912 |
| Closing, 5 x 10  | 0.806 |   0.883   |  0.902 |
| Closing, 11 x 2  | 0.815 |   0.897   |  0.899 |
| Closing, 11 x 5  | 0.801 |   0.875   |  0.904 |
| Closing, 11 x 10 | 0.777 |   0.842   |  0.910 |
| Opening, 11 x 5  | 0.858 |   0.984   |  0.871 |
| Op. 11x5, Cl. 11x5 | 0.861 | 0.981   |  0.875 |
| Op. 13x5, Cl. 13x5 | 0.855 | 0.983   |  0.868 |
| Op. 11x5, Cl. 7x10 | 0.862 | 0.981   |  0.876 |
| Op. 11x5, Cl. 7x20 | 0.862 | 0.977   |  0.880 |